In [ ]:
import os
import pandas as pd
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### Import csv

In [ ]:
df = pd.read_csv("argus_info_output_fluidics.csv")
df['added'] = False

In [ ]:
df.head()

### Populate nested dictionary with times and subtasks of each higher level device task

In [ ]:
# A recursive function to record and notate the lineage and times of a nested step within a fluidics step
# Makes a new dict layer if it does not exist and populates with the data being passed to it

def add_layer_to_dict(current_dict, lineage_array, lineage_time, lineage_added, lineage_sum_time):
    # Pop from the first index position of dictionary to recusively ingest the entire data file
    current_layer = lineage_array.pop(0)
    current_time = lineage_time.pop(0)
    current_added = lineage_added.pop(0)
    current_sum_time = lineage_sum_time.pop(0)
    
    # Check if step exists at the current nested layer of the dictionary
    # Append information if exists
    if current_dict.get(current_layer) != None:
        # Check if the specific row has already been added to the dictionary
        if current_added == False:
            current_dict[current_layer]['times'].append(current_time)
            current_dict[current_layer]['sum_times'].append(current_sum_time)
            
    # If not, create that layer and append information
    else:
        current_dict[current_layer] = {'times': [], 'sum_times': [], 'substeps': {}}
        current_dict[current_layer]['times'].append(current_time)
        current_dict[current_layer]['sum_times'].append(current_sum_time)
        
    # While there are still layers in the dictionary, call the function again with the next layer down in the dict
    if len(lineage_array) > 0:
        add_layer_to_dict(current_dict[current_layer]['substeps'], lineage_array, lineage_time, lineage_added, lineage_sum_time)
    else:
        return

In [ ]:
# Create empty dict
top_level = {}

# Iterate through main dataframe
for i, row in enumerate(df.iterrows()):
    row = row[1]
    layer = row['layer']
            
    # Check if the current row is the end of a device task chain
    if row['has_doughter'] == False:
        step_type = row['type']
        layer = row['layer']
        lineage_array = []
        lineage_time = []
        lineage_sum_time = []
        lineage_added = []
        lineage_array.append(row['type'])
        lineage_time.append(row['execution_time'])
        lineage_sum_time.append(row['duration'] - row['incubation_duration'])
        lineage_added.append(row['added'])
        df.iloc[[i],[11]] = True
        parent_row_iloc = int(row['parent'])
        
        # Ascend back up the device task chain until you rach the top level task
        while layer != 0:
            parent_type = df.iloc[parent_row_iloc]['type']
            layer = df.iloc[parent_row_iloc]['layer']
            exec_time = df.iloc[parent_row_iloc]['execution_time']
            daughter_time = df.iloc[parent_row_iloc]['duration'] - df.iloc[parent_row_iloc]['incubation_duration']
            added_state = df.iloc[parent_row_iloc]['added']
            df.iloc[[parent_row_iloc],[11]] = True
            try:
                parent_row_iloc = int(df.iloc[parent_row_iloc]['parent'])
            except ValueError:
                pass #Reached a NaN from top layer
            lineage_array.insert(0, parent_type)
            lineage_time.insert(0, exec_time)
            lineage_added.insert(0, added_state)
            lineage_sum_time.insert(0, daughter_time)
        
        # Add the device task stack to the dictionary
        add_layer_to_dict(top_level, lineage_array, lineage_time, lineage_added, lineage_sum_time)
        
top_level_nested = {'times': [0], 'sum_times': [0], 'substeps': top_level}
# Reset the dataframe column to false to allow the block to be rerun without error
df['added'] = False

In [ ]:
# Function to determin statistics for each layer of the nested dictionary
def nested_dict_pairs_iterator(current_dict):
    try:
        current_dict['stats'] = []
    except TypeError:
#         print("hit the type error")
        return
    
    keys = current_dict.keys()
    
    # Check if the current layer contains a substepd dict. If not, it is the top level and must be treated differently
    if 'substeps' in keys:
        current_dict['stats'].append(np.mean(current_dict['times'])) # step time mean
        current_dict['stats'].append(np.std(current_dict['times'])) # step time std
        current_dict['stats'].append(np.mean(current_dict['sum_times'])) # aggregate time mean
        current_dict['stats'].append(np.std(current_dict['sum_times'])) # aggregate time std
        current_dict['stats'].append(len(current_dict['times'])) # Number of times completed in run

        keys = current_dict['substeps'].keys()
    
        if keys:
            for key in keys:
                nested_dict_pairs_iterator(current_dict['substeps'][key])
        else:
            return
    else:
        if keys:
            for key in keys:
                nested_dict_pairs_iterator(current_dict[key])
        else:
            return
        
# Call function on top_level dict
nested_dict_pairs_iterator(top_level)
nested_dict_pairs_iterator(top_level_nested)
del top_level_nested['substeps']['stats']

### Plotting

In [ ]:
# Function to plot the high level historgram of the top layer of the dictionary
# as well as a scatter plot with error bars of the next layer of subtasks
# which comprise the top layer task

def plot_times(plot_dict, title=''):
    mean_times = []
    std_times = []
    mean_sum_times = []
    std_sum_times = []
    num_times = []
    steps = []

    for key in list(plot_dict['substeps'].keys()):
        stats = plot_dict['substeps'][key]['stats']
        steps.append(key)
        mean_times.append(stats[0])
        std_times.append(1.96*stats[1])
        mean_sum_times.append(stats[2])
        std_sum_times.append(1.96*stats[3])
        num_times.append(stats[4])
        
        
    if plot_dict['stats'][0] > 1:
        steps.append('wait_s')
        mean_times.append(plot_dict['stats'][0])
        std_times.append(plot_dict['stats'][1])
        mean_sum_times.append(plot_dict['stats'][0])
        std_sum_times.append(plot_dict['stats'][1])
        num_times.append(plot_dict['stats'][4])

    print(plot_dict['stats'])
    print(steps)
    print(list(zip(steps, num_times)))

    fig, ax = plt.subplots(figsize=(10,6))
    ax.hist(plot_dict['sum_times'])
    ax.set(title='Timing Distributions for {}'.format(title))
    plt.show()

    fig, ax = plt.subplots(figsize=(10,6))
    ax.scatter(steps, mean_sum_times)
    ax.set(title='Average substep times and distribution for {}'.format(title), 
          xlabel='Substep', ylabel='Time (s)')
    bottom, top = plt.ylim()
    offset = top*0.05
    for index in range(len(steps)):
          ax.text(steps[index], mean_sum_times[index]+offset, 
                  "{} times, \n{} mins".format(num_times[index], int((num_times[index] * mean_sum_times[index])/60)), 
                  size=12)
    plt.errorbar(steps,mean_sum_times,yerr=std_sum_times, linestyle="None")
    plt.xticks(rotation = 45)
    plt.show()

### Plotting Scratch

In [ ]:
plot_dict = top_level['HURWellExchange']['substeps']['HURDispense']
plot_times(plot_dict, 'HURWellExchange')

In [ ]:
plot_dict = top_level['HURWellExchange']['substeps']['HURDispense']['substeps']['WellExtract']
plot_times(plot_dict, 'WellExtract')

In [ ]:
plot_dict = top_level['HURWellExchange']['substeps']['HURDispense']['substeps']['WellExtract']['substeps']['AlignSipperTubeToWellExtractLocation']
plot_times(plot_dict, 'WellExtract AlignSipperTube')

In [ ]:
plot_dict = top_level['HURWellExchange']['substeps']['HURDispense']['substeps']['WellExtract']['substeps']['MachineVisionCorrection']
plot_times(plot_dict, 'HURWellExchange WellExtract MachineVisionCorrection')

In [ ]:
plot_dict = top_level['LURWellExchange_DryTimeOptimized']
plot_times(plot_dict, 'LURWellExchange_DryTimeOptimized')

In [ ]:
plot_dict = top_level['LURWellExchange_DryTimeOptimized']['substeps']['WellExtract']
plot_times(plot_dict, 'LURWellExchange_DryTimeOptimized WellExtract')

In [ ]:
plot_dict = top_level['LURWellExchange_DryTimeOptimized']['substeps']['LURDispenseAspiratedReagent']
plot_times(plot_dict, 'LURWellExchange_DryTimeOptimized LURDispenseAspiratedReagent')

In [ ]:
plot_dict = top_level_nested
plot_times(plot_dict, 'Run')

In [ ]:
plot_dict = top_level_nested['substeps']['CleanObjective']
plot_times(plot_dict, 'Clean Objective')

In [ ]:
plot_dict = top_level_nested['substeps']['CleanObjective']['substeps']['OWCExtract']
plot_times(plot_dict, 'CleanObjective OWCExtract')

In [ ]:
plot_dict = top_level_nested['substeps']['CleanObjective']['substeps']['OWCExtract']['substeps']['AlignSipperTubeToOWCWashReservoir']
plot_times(plot_dict, 'CleanObjective OWCExtract')